# FWI diagnostics in Jupyter Notebooks

Fullwave Steering Group Meeting, 16 October 2019

#### Kajetan Chrapkiewicz

# About my PhD

3D FWI of the active **Santorini volcano** (large caldera-forming eruptions in the past)
- very sparse data
- long offsets up to around 100 km

# Table of contents

#### 1. Jupyter Notebooks

#### 2. FullwavePy interface

#### 3. Three simple examples of FWI diagnostics

#### 4. Note for code developers

# Jupyter Notebooks

## Philosophy

### Cells

In [7]:
print('You can split execution into separate...')

You can split execution into separate...


In [8]:
print('...steps')

...steps


### Blend code, plots, notes, ...

This is a cell with a note formatted with **markdown/HTML**.

In [10]:
x = 5 # this is a Python3 cell

## Features

### Powerful

In [9]:
!echo 'For example, you can run bash commands'

For example, you can run bash commands


![title](figs/jupyter.png)

### Easy to install

Install IPython and Jupyter with [conda](https://www.anaconda.com/download):

    conda install ipython jupyter

Launch in the browser:

    jupyter notebook

### Shareable

For example as a presentation like this one. It was created from a notebook with a single command:

In [ ]:
jupyter nbconvert nb.ipynb --to slides --post serve

### Popular

Tool of choice in 'data-science' community. Also increasingly common in seismology.

![title](figs/igel.png)

# FullwavePy interface

- a small, object-oriented Python3 package
- a helpful tool to keep track of, plot and modify Fullwave's input and output
- in above sense, an interface between Fullwave3D and Jupyter

# Example 1
a synthetic project created from scratch

## Configure the notebook

Load all the defaults with a single line:

In [ ]:
%load ~/software/fullwavepy/nb_config.py

This will produce a cell containing following lines editable while running the notebook:

In [ ]:
from fullwavepy import * # load basic functions and classes

In [ ]:
plt.style.use(['ggplot', 'fullwavepy']) # set plotting style

In [ ]:
set_logging_level(INFO) # TRACE / DEBUG / INFO / WARNING / ERROR / CRITICAL

## Generate synthetics

### Initialize project

#### Set paths

In [ ]:
exe = {'fullwave': '~/fullwave3D/rev690/bin/fullwave3D.exe' # run via PBS
       'segyprep': '~/segyprep/bin/segyprep.exe', # run locally
      }

#### Set project geometry

In [ ]:
nx = 101    # no. of in-line nodes
ny = 61     # no. of in-line nodes
nz = 31     # no. of in-line nodes
dx = 50     # grid cell-size [m]
dt = 0.001  # time-step [s]
ns = 2000   # no. of time-samples

#### Create a project-class instance

In [ ]:
proj = ProjSyn('example01_syn', io='sgy', dims=[nx,ny,nz], dx=dx, 
               dt=dt, ns=ns, exe=exe)

### Prepare input

#### Prepare a true model of vp

In [ ]:
proj.inp.truevp.prepare(vel=3000) # vel - homog. vel. [m/s]

#### Prepare a source wavelet

In [ ]:
proj.inp.rawsign.prepare(shape='ricker', fpeak=5) # fpeak - peak freq. [Hz]

#### Prepare a SegyPrep parameter file

In [ ]:
sp = {'geometry': 'regular', 
      'geometry_in_nodes': True,
      'soux0': nx//2, 'soudx': 2, 'sounx': 1, 
      'souy0': ny//2, 'soudy': 2, 'souny': 1, 
      'souz':  nz//2, 
      'recx0': 2, 'recdx': 4, 'recnx': 100, 
      'recy0': 3, 'recdy': 4, 'recny': 100, 
      'recz':  nz//2} 

In [ ]:
proj.inp.sp.prepare(**sp, cat=0)

#### Run SegyPrep

In [ ]:
proj.inp.sp.run(cat=0)

#### Prepare Fullwave's runfile

In [ ]:
runfile = {'b_abs': 20, 'e_abs': 30} # absorbing boundaries
proj.inp.runfile.prepare(**runfile, cat=0)

#### Check Fullwave's input

In [ ]:
proj.inp.check()

#### Prepare a bash script for local runs

In [ ]:
proj.inp.bash.prepare(ompthreads=8, cat=0)

### Run Fullwave

In [ ]:
proj.run(timer=1, cat=0)

## Invert synthetics

### Initialize project

In [ ]:
proj = ProjInv('example01_inv', exe=exe)

### Prepare input

#### Duplicate the synthetic-project input

In [ ]:
proj.inp.dupl('./example01_syn/inp/', 'example01_syn')

#### Overwrite some files

In [ ]:
proj.inp.startvp.prepare(vel=8000) # homogeneous velocity [m/s]
proj.inp.sp.prepare(**sp, cat=0)
proj.inp.sp.run(cat=0)
proj.inp.runfile.prepare(**runfile, blocks=[{'freq': 3.0, 'nits': 1}], cat=0)
proj.inp.bash.prepare(ompthreads=8, cat=0)

### Run

In [ ]:
proj.run(timer=1, cat=0)

### Plot output

In [ ]:
proj.out.dumpcomp.it[1][source_id=1].plot_phase(freq=[2,4])

![title](figs/f2.png)

![title](figs/f4.png)

# Example 2
synthetic project already created - Marmousi2.5D

## Generate synthetics

### Initialize project

In [ ]:
proj = ProjSyn('example02_syn', exe=exe)

### Prepare input

#### Duplicate one of the Fullwave's test-case projects

In [ ]:
proj.inp.dupl('./Fullwave3D-testcases-20120910/Marmousi-2.5D-acoustic-iso-time/', 
              'MarmAIT')

#### Create a PBS script to run Fullwave on a cluster

In [ ]:
proj.inp.pbs.prepare(hours=2, select=16, mpiprocs=8, q='general')

## Invert synthetics

### Initialize project

In [ ]:
proj = ProjInv('example02_inv', exe=exe)

### Prepare input

#### Duplicate the synthetic-project input

In [ ]:
proj.inp.dupl('./example02_syn/inp/', 'example02_syn')

#### Overwrite some parameters

In [ ]:
proj.inp.runfile.modify(problem='inversion')

#### Set environment variables

In [ ]:
proj.env['SLAVES_DUMPCSREFS'] = '1,91,185'
proj.env['SCHEDULER_DUMPGRAD'] = 'yes'

#### Create a PBS script to run Fullwave on a cluster

In [ ]:
proj.inp.pbs.prepare(hours=24, select=16, mpiprocs=8, q='general')

### Plot output

In [ ]:
proj.out.vp.plot_all()

![title](figs/ex2_VP.gif)

In [ ]:
proj.out.grad.it[6].plot()

![title](figs/grad_006.png)

In [ ]:
proj.out.functional.plot()

![title](figs/example2_func.png)

![title](figs/iters2.gif)

In [ ]:
proj.out.dumpcomp.it[:][shot_id=1].plot_phase(freq=4) 

![title](figs/ex2_freqs.gif)

# Example 3
3D field-data example - Santorini

## Generate synthetics

### Initialize project

#### Set paths

In [ ]:
truevp = 'startmods/jm_inversecheck-StartVp.sgy'
rawsign = 'wavelets/wavelet_19-09-22.sgy'
topo = 'surfaces/bathy_x_-8e4_8e4_y_-4e4_4e4_cell_50.vtr'
srcs = 'sources/all-Sources.csv'
recs = 'receivers/all-Receivers.csv'
data_path = '/home/kmc3817/heavy_PhD/DATA/Santorini_2015/seismic/OBS/segy_local_coords/'
data_files = get_files(data_path, '*_4.sgy')

#### Set project geometry

In [ ]:
dt = 0.0025  
ns = 2000    
dx = 50      
# Local coordinates [m]
x1 = 8e3     
x2 = 2e4    
y1 = 1e3    
y2 = 15e3    
z1 = 0
z2 = 4000    
box = [x1, x2, y1, y2, z1, z2]

#### Create a project-class instance

In [ ]:
proj = ProjSyn('example3_syn', io='sgy', box=box, dx=dx, dt=dt, ns=ns,
               info='Generating synthetics for model and data QC')

#### Plot acquisition geometry

In [ ]:
proj.geom.plot(topo=topo, srcs=srcs)

![title](figs/example3_box.png)

### Prepare input

In [ ]:
proj.inp.truevp.prepare(copy=truevp)
proj.inp.rawsign.prepare(copy=rawsign)
proj.inp.rawseis.prepare(data_files, cat=0)
proj.inp.sp.prepare(geometry='sgy', reciprocity=True)
proj.inp.runfile.prepare()
proj.inp.pbs.prepare(select=2, mpiprocs=8)

### Plot output

![title](figs/example3_ileave.png)

![title](figs/example3_phase_obs.png)

![title](figs/example3_phase_dif.png)

# For developers

- object-oriented
- (auto)logging